In [1]:
# dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import numpy as np

In [2]:
# SQL dependencies
# sqlite database file
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

engine = create_engine("sqlite:///kimarite.sqlite")

In [3]:
sumo_results_df = pd.DataFrame()

In [4]:
# read in csv files and create dataframe
for num in range(1983,2019):
    df = pd.read_csv("sumo_results/" +str(num)+".csv")
    sumo_results_df = sumo_results_df.append(df)


In [5]:
sumo_results_df.head()

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win
0,1983.01,1,4140,J13w,Chikubayama,0-1 (7-8),False,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),True
1,1983.01,1,4306,Ms1e,Ofuji,1-0 (6-1),True,yorikiri,4140,J13w,Chikubayama,0-1 (7-8),False
2,1983.01,1,1337,J12w,Tochitsukasa,1-0 (9-6),True,oshidashi,4323,J13e,Shiraiwa,0-1 (3-12),False
3,1983.01,1,4323,J13e,Shiraiwa,0-1 (3-12),False,oshidashi,1337,J12w,Tochitsukasa,1-0 (9-6),True
4,1983.01,1,4097,J12e,Tamakiyama,0-1 (8-7),False,yorikiri,4319,J11w,Harunafuji,1-0 (5-10),True


In [6]:
# Pull out only kimarite column
kimarite = sumo_results_df[['kimarite']]

In [7]:
kimarite.head()

,kimarite
0,yorikiri
1,yorikiri
2,oshidashi
3,oshidashi
4,yorikiri


In [8]:
# Pull only unique values
kimarite_less = kimarite.drop_duplicates()


In [9]:
# Reset the index
kimarite_less = kimarite_less.reset_index(drop=True)
kimarite_less.head()

,kimarite
0,yorikiri
1,oshidashi
2,tsukidashi
3,hatakikomi
4,yoritaoshi


In [10]:
# Drop/reset index to use new index as new column
kimarite_keys = kimarite_less.reset_index(level=0)

In [11]:
kimarite_keys.head()

,index,kimarite
0,0,yorikiri
1,1,oshidashi
2,2,tsukidashi
3,3,hatakikomi
4,4,yoritaoshi


In [12]:
# Rename column to kimarite_id to create key for relational database
kimarite_keys = kimarite_keys.rename(columns={'index':'kimarite_id'})
kimarite_keys.head()

,kimarite_id,kimarite
0,0,yorikiri
1,1,oshidashi
2,2,tsukidashi
3,3,hatakikomi
4,4,yoritaoshi


In [13]:
# Wikipdeia article with kimarite descriptions and groupings
url='https://en.wikipedia.org/wiki/Kimarite'
res = requests.get(url)

In [14]:
# BeautifulSoup
soup = bs(res.text, 'html.parser')

In [15]:
# Dataframe to match up kimarite and the type/common categories
# Dataframe to match up kimarite and the text description
# Use same dataframe but add categories first
kimarite_desc = pd.DataFrame()

In [16]:
# Pull table with top grouping
categories = soup.select('li.toclevel-1')

In [17]:
# Find out number of categories
# There are two categories that do not have any items, see also/external links part of wikipedia navigation
len(categories)

9

In [18]:
# Find list of kimarite that fall under each category
# Category name is the first list item
list = categories[0].find_all('i')

In [19]:
list[0].text

'Kihonwaza'

In [20]:
counter = 0

# For loop to put the info into dataframe
for num in np.arange(0,7):
    list = categories[num].find_all('i')
    # Set category name variable on first row item
    category = list[0].text
    for row in list[1:]:
        kimarite_desc.loc[counter,'category'] = category
        kimarite_desc.loc[counter, 'kimarite'] = row.text
        counter += 1
        
        
        

In [21]:
kimarite_desc.head()

,category,kimarite
0,Kihonwaza,Abisetaoshi
1,Kihonwaza,Oshidashi
2,Kihonwaza,Oshitaoshi
3,Kihonwaza,Tsukidashi
4,Kihonwaza,Tsukitaoshi


In [22]:
# Checking table values
kimarite_desc.describe()

,category,kimarite
count,87,87
unique,7,87
top,Tokushuwaza,Yoritaoshi
freq,19,1


In [23]:
description = soup.find_all(['h3','p'])

In [24]:
description

[<p><i><b>Kimarite</b></i><span style="font-weight: normal"> (<span lang="ja">決まり手</span><span style="display:none">,</span> <i>kimari-te</i>)</span> are winning techniques in a <a href="/wiki/Sumo" title="Sumo">sumo</a> bout. For each bout in a Grand Sumo tournament (or <i><a href="/wiki/Honbasho" title="Honbasho">honbasho</a></i>), a sumo referee, or <i><a href="/wiki/Gy%C5%8Dji" title="Gyōji">gyōji</a></i>, will decide and announce the type of <i>kimarite</i> used by the winner. It is possible (although rare) for the <a href="/wiki/Judge_(sumo)" title="Judge (sumo)">judges</a> to modify this decision later. Records of the <i>kimarite</i> are kept and statistical information on the preferred techniques of different wrestlers can be deduced easily. For example, a <a href="/wiki/Pie_chart" title="Pie chart">pie chart</a> of the <i>kimarite</i> used by each <i><a href="/wiki/Sekitori" title="Sekitori">sekitori</a></i> in the past year can be found on the <a href="/wiki/Japan_Sumo_Associ

In [25]:
# Lists to hold kimarite and text descriptions
kimarite_list = []
descriptions_list = []

# For loop to add description to table
saved = 'none'

for row in description[4:]:
    # Add kimarite description to variable
    if row.name == 'h3':
        saved = row.text
    # Use variable to grab the next paragraph description from wikipedia
    elif row.name == 'p':
        for kimarite in kimarite_desc['kimarite']:
            if kimarite in saved:
                print(kimarite)
                kimarite_list.append(kimarite)
                print(row.text)
                descriptions_list.append(row.text)
            
            

Abisetaoshi
Forcing down the opponent on their back by leaning forward while in a grappling position (backward force down).

Oshidashi
Pushing the opponent out of the ring without holding their mawashi or belt, nor fully extending his arms.  Hand contact must be maintained through the push (front push out).

Oshitaoshi
Pushing the opponent down out of the ring (the opponent falls out of the ring instead of backing out) without holding their mawashi. Hand contact is maintained throughout the push (front push down).

Tsukidashi
Thrusting the opponent backwards out of the ring with one or a series of hand thrusts. The attacker does not have to maintain hand contact (front thrust out).

Tsukitaoshi
Thrusting the opponent down out of the ring (the opponent falls over the edge) onto their back with a hard thrust or shove (front thrust down).

Yorikiri
Maintaining a grip on the opponent's mawashi, the opponent is forced backwards out of the ring (front force out).

Yoritaoshi
Maintaining a gr

In [26]:
# Combine two lists
kimarite_temp = pd.DataFrame({'kimarite':kimarite_list,'description':descriptions_list})

In [27]:
# Clean up any double values
# Used text from Wikipedia to find correct columns
kimarite_temp['kimarite'].value_counts()

# Duplicates
pd.concat(g for _, g in kimarite_temp.groupby("kimarite") if len(g) > 1)

,kimarite,description
30,Mitokorozeme,\n
31,Mitokorozeme,A triple attack. Wrapping one leg around the o...
32,Mitokorozeme,"This is a very rare technique, first used in t..."
68,Tsutaezori,Shifting the extended opponent's arm around an...
69,Tsutaezori,Special techniques.\n
41,Watashikomi,"While against the ring of the surface, the att..."
42,Watashikomi,Twist down techniques.\n
20,Yaguranage,With both wrestlers grasping each other's mawa...
21,Yaguranage,Leg tripping techniques.\n
88,Yobimodoshi,Reacting to the opponent's reaction to the att...


In [28]:
# Using dataframe, drop duplicates from kimarite_temp df
locs = [30,32,69,42,21,89,7,62]

# Drop rows
kimarite_temp_clean = kimarite_temp.drop(kimarite_temp.index[locs])

In [29]:
# Check that there are no more duplicates
kimarite_temp_clean.describe()

,kimarite,description
count,87,87
unique,87,87
top,Yoritaoshi,"While wrestlers face each other, to pick up th..."
freq,1,1


In [31]:
# Join descriptions to grouping index
kimarite_desc = kimarite_desc.merge(kimarite_temp_clean, on='kimarite')

In [32]:
kimarite_desc.head()

,category,kimarite,description
0,Kihonwaza,Abisetaoshi,Forcing down the opponent on their back by lea...
1,Kihonwaza,Oshidashi,Pushing the opponent out of the ring without h...
2,Kihonwaza,Oshitaoshi,Pushing the opponent down out of the ring (the...
3,Kihonwaza,Tsukidashi,Thrusting the opponent backwards out of the ri...
4,Kihonwaza,Tsukitaoshi,Thrusting the opponent down out of the ring (t...


In [45]:
# Join on kimarite_keys index from datafile, see if all kimarite in dataframe have descriptions/types
# Lowercase kimarite in kimarite_desc dataframe
kimarite_desc['kimarite'] = kimarite_desc['kimarite'].str.lower()
kimarite_final = kimarite_keys.merge(kimarite_desc, on='kimarite', how='left')

In [46]:
# Double-check no Nan values
kimarite_final.count()

kimarite_id    83
kimarite       83
category       81
description    81
dtype: int64

In [50]:
# Find Nan values
null_kimarite = kimarite_final[kimarite_final.isna().any(axis=1)]
null_kimarite

fusen = A sumo match that is cancelled due to the absence of one of the rikishi.
hansoku = (losing techniques) Violation of sumo rules: These result in automatic loss of the person who did it:

,kimarite_id,kimarite,category,description
19,19,fusen,NaN,NaN
56,56,hansoku,NaN,NaN


In [51]:
# Googled empty values
# Added empty results - 
# fusen = 
    # Category - No-Show
    # A sumo match that is cancelled due to the absence of one of the rikishi.
kimarite_final.loc[19,'category'] = 'No-Show'
kimarite_final.loc[19,'description'] = 'A sumo match that is cancelled due to the absence of one of the rikishi.'
# hansoku
    # Category - Losing Techniques
    # Description - Violation of sumo rules - automatic loss of the person who did it.
kimarite_final.loc[56,'category'] = 'Losing Techniques'
kimarite_final.loc[56,'description'] = 'Violation of sumo rules - automatic loss of the person who did it.'

In [52]:
kimarite_final.count()

kimarite_id    83
kimarite       83
category       83
description    83
dtype: int64

In [53]:
# Finished dataframe to sqlite file/database
kimarite_final.to_sql('kimarite', con=engine)